In [1]:
import pandas as pd
import numpy as np
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import ast

# Activer la précision mixte pour accélérer les calculs
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

c:\Users\ilyes\OneDrive\Bureau\M2-TextMining\datamining\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


**Chargement des données**

In [2]:
path = "./archive/lemmetised_clean_data.csv"
df = pd.read_csv(path, delimiter=",", quotechar='"')

**Séléction des tags et réduction du Dataset**


In [3]:
df['tags'] = df['tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Sélectionner les 50 tags les plus fréquents (ajustable)
all_tags = [tag for tag_list in df['tags'] for tag in tag_list]
tag_counts = pd.Series(all_tags).value_counts()
num_top_tags = min(15, len(tag_counts))  # Modifier pour augmenter/diminuer
selected_tags = tag_counts.head(num_top_tags).index.tolist()

df = df[df['tags'].apply(lambda tags: any(tag in selected_tags for tag in tags))]
df['main_tag'] = df['tags'].apply(lambda tags: next((tag for tag in tags if tag in selected_tags), None))
df.dropna(subset=['clean_text'], inplace=True)

print(f"Taille du dataset après filtrage: {df.shape}")

Taille du dataset après filtrage: (64571, 8)


**Préparation des textes et etiquettes et encodage des étiquettes**

In [4]:
# Préparation des textes et des étiquettes
texts = df['clean_text'].astype(str).tolist()
labels = df['main_tag'].tolist()

# Encodage des étiquettes
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
num_classes = len(np.unique(y))

**Tokenization des textes**

In [ ]:
# Chargement du tokenizer DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Réduire la séquence pour accélérer
max_len = 32  

# Tokenisation des textes
tokens = tokenizer(texts, max_length=max_len, padding=True, truncation=True, return_tensors="np")

c:\Users\ilyes\OneDrive\Bureau\M2-TextMining\datamining\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ilyes\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


**Division des données en train/test**

In [6]:
# Convertir les tokens en tableaux NumPy
X = np.array(tokens['input_ids'])  # Convertir les IDs en NumPy array
y = np.array(y)  # Convertir les labels en NumPy array

# Division train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Création des datasets TensorFlow
batch_size = 64  # Augmentation de la taille des batchs
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size).prefetch(tf.data.AUTOTUNE)


**Création du modèle Transformer (BERT)**

In [7]:
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_classes)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

**Compilation**

In [8]:
optimizer = Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

**Définition d'un Early Stopping**

In [9]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

**Entraînement**

In [10]:
model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=5,  # Ajustable
    callbacks=[early_stopping]
)

Epoch 1/5


808/808 [==============================] - 6895s 9s/step - loss: 1.6224 - accuracy: 0.4712 - val_loss: 1.4053 - val_accuracy: 0.5254
Epoch 2/5
808/808 [==============================] - 6741s 8s/step - loss: 1.3156 - accuracy: 0.5575 - val_loss: 1.3800 - val_accuracy: 0.5335
Epoch 3/5
808/808 [==============================] - 6765s 8s/step - loss: 1.1769 - accuracy: 0.6033 - val_loss: 1.4073 - val_accuracy: 0.5302
Epoch 4/5
808/808 [==============================] - 6645s 8s/step - loss: 1.0326 - accuracy: 0.6500 - val_loss: 1.4608 - val_accuracy: 0.5266
Epoch 5/5
808/808 [==============================] - 6788s 8s/step - loss: 0.8962 - accuracy: 0.6935 - val_loss: 1.5650 - val_accuracy: 0.5198


**Évaluation**

In [11]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Accuracy sur les données de test: {accuracy:.4f}")

202/202 [==============================] - 593s 3s/step - loss: 1.3800 - accuracy: 0.5335
Accuracy sur les données de test: 0.5335
